<a href="https://colab.research.google.com/github/estbasili/BD-ML/blob/main/Laboratorio_1_ML%26BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#         **LABORATORIO 1 ML&BD**

## Carga, limpieza y preparacion de datos

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_de_archivos ='/content/drive/MyDrive/UPSO/ML/Practico'

In [ ]:
# genero una lista con todos los archivos que estan dentro de la carpeta
# ['Dataset_de_ventas_Septiembre.csv',..,'Dataset_de_ventas_Abril.csv']

archivos = os.listdir(ruta_de_archivos)

# genero un dataframe donde voy a guardar los datos de los meses

dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'): # filtro solo los archivos CSV
    mes = archivo.split('_')[3].split('.')[0] # obtengo de: Dataset_de_ventas_Septiembre.csv solo Septiembre
    ruta_completa = os.path.join(ruta_de_archivos, archivo) # obtengo la ruta completa al archivo
    # carga el archivo CSV en un dataframe y hago copia, ejemplo el del mes de Septiembre
    df = pd.read_csv(ruta_completa)
    df_ventas= df.copy()

    # limpieza de campos

    #El errors = 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'],errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    # Elimino Nan y me quedo solo con los numericos
    df_ventas = df_ventas.dropna()

    # transformo los valores numericos al tipo de dato int
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)
    # Agregar el DataFrame al diccionario usando el nombre del mes
    # dataframe = {key = 'mes': valor = contenido de mes.csv}
    dataframes[mes] = df_ventas


#Fin de limpieza
#En el dict dataframes tengo todos los meses y sus valores limpios


Funciones

In [ ]:
def extraer_datos(datos):
  #transformo el campo Fecha de Pedido en tipo datetime e incorporo una columna con la hora y el dia
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Nombre Dia'] = datos['Fecha de Pedido'].dt.day_name()##### esta modificacion 11/09 para saber el nombre del dia para el punto 6
  return datos

In [ ]:
def extraer(data):
  # separo la columna Direccion de envio en direccion, Ciudad y Estado
  data[['Direccion','Ciudad','CP']]= data['Dirección de Envio'].str.split(', ', expand=True)
  return data

## 1. Ventas Totales / Ingresos por mes

In [ ]:
# creo listas para almacenar los totales de ventas, ingresos generados y los nombres de los meses
totales_ventas = []
ingresos_generados = []
ingresos_generados_texto=[] # esta lista es para poder visulizar el texto de las barras con el signo "$"
nombres_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in nombres_meses:
  dataframe_mes = dataframes[mes]
  total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum() # sumo todas las ventas del mes
  ingresos_generados_mes = (dataframe_mes['Precio Unitario'] * dataframe_mes['Cantidad Pedida']).sum() #ingresos total del mes
  totales_ventas.append(total_ventas_mes)
  ingresos_generados.append(ingresos_generados_mes)
  ingresos_generados_texto.append("$" + str(ingresos_generados_mes))


# Creo objeto figura con el grafico de barras con los totales de las ventas mensuales

fig = go.Figure()
fig.add_trace(go.Bar(
    x = nombres_meses,
    y = totales_ventas,
    marker = dict(color = totales_ventas, colorscale = 'RdYlGn', colorbar = dict( title = "Total de ventas", x = 1.0, thickness = 15) ),
    text = totales_ventas,
    textposition = 'auto',
))

# configuracion de los ejes y titulos

fig.update_layout(
    title =" Totales de ventas por mes",
    xaxis = dict( title = "Mes" ),
    yaxis = dict( title = "Total de ventas"),
)

fig.show()




1. Comportamiento de las ventas en los distintos meses:

● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

Rta: Se puede apreciar que de Enero a Abril tiene una tendencia creciente. Seguida de una tendencia levemente decreciente hacia el mes de Septiembre.Recuperandose en Octubre, volviendo a decaer en Noviembe y cerrando el año con tendencia creciente.

● ¿Hubo algún mes que se destacó en términos de ventas?

Rta: El mes de Diciembre con un total de 28.121 de productos vendidos. Correspondiente con las fiestas de fin de año.

● ¿Cuál es el ingreso total generado por mes?

Rta: Se puede apreciar en el siguiente grafico.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x = nombres_meses,
    y = ingresos_generados,
    marker = dict(color =  ingresos_generados, colorscale = 'RdYlGn', colorbar = dict( title = "Ingresos de mes", x = 1.0, thickness = 15) ),
    text =  ingresos_generados_texto,
    textangle = 0, # esto es para colocar en horizontal ( 0 grados) el texto en cada barra
    textposition = 'auto',
))

# configuracion de los ejes y titulos

fig.update_layout(
    title =" Ingresos totales por mes",
    xaxis = dict( title = "Mes" ),
    yaxis = dict( title = "Total de Ingresos"),
)

fig.show()

## 2. Optimizacion / Patron de ventas por hora

In [ ]:
# genero una  lista Horas con las horas en formato de 24hs

Horas = list (range(24))

fig = go.Figure()

for mes in nombres_meses:  # recorro los dataframe por mes y separo las horas y los dias en columnas nuevas
   dataframe_mes= dataframes[mes]
   dataframe_mes = extraer_datos(dataframe_mes)
   Ventas_por_hora = dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()#aqui deberia haber un sum() ya que se necesita sumar las cantidades pedidas.

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
   fig.add_trace(go.Scatter(
       x=Horas,
       y=Ventas_por_hora,
       mode='lines',
       name= mes,
       line=dict(width=2,shape='spline'),
   ))


# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()



2. Optimización de la publicidad y patrón de ventas por hora:

● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
probabilidad de compra?

Rta: Se podria decir que el momento es alrededor de las 12hs y las 19hs.
Teniendo en algunos meses mayor ponderacion las 19hs.

● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

Rta: Se puede apreciar que los patrones de ventas por hora a lo largo del año son semenjantes.
Siendo Enero el mes de menor unidades vendidas por hora. Seguido de
         Febrero, Septiembre y agosto con un promedio mayor que Enero.
         Abril, noviembre y Mayo mayor que Febrero. Y los meses de Octubre y Diciembre que lideran en unidades vendidas por hora de los meses ante dicho.




In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [19, 12, 16]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in nombres_meses:
        dataframe_mes = dataframes[mes]
        dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=nombres_meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()



● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
en los distintos meses?

Rta: Tomando las curvas de unidades vendidas de las 16hs y 19hs se puede apreciar que los patrones son similares. Teniendo un crecimiento desde el mes de Enero a Abril. Con 554 unidades a las 16 hs y 657 a las 19hs en Enero y de 1054 unidades a las 16hs y 1286 a las 19hs en Abril.
Decreciendo desde el mes de Abril a Septiembre, llegando a 622 unidades a las 16hs y 793 a las 19hs. Volviendo a tener una tendencia crecien desde Septiembre a Diciembre, llegando a 1335 unidades a las 16hs y 1791 a las 19hs.
Caso particular es la curva de unidades vendidas de las 12hs. Vemos que esta tiene similiar comportamiento, solo difiere en que el periodo de crecimiento de Enero a Abril que se extiende hasta el mes de Mayo. Despues de dicho mes, se puede decir que sigue un comportamiento similiar a los patrones de unidades vendidas de las 16hs y 19hs.

## 3. Distribucion de ventas por Ubicacion

In [ ]:
fig = go.Figure()

for mes in nombres_meses:
  dataframe_mes=dataframes[mes]
  dataframe_mes = extraer(dataframe_mes)
  df_Estado = dataframe_mes['CP'].str.split(' ', expand=True)
  df_Estado.columns=['Estado','Numero']
  df_desglosado = pd.concat([dataframe_mes,df_Estado],axis=True)
  ventas_mes_Estado = df_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
  ventas_mes_Estado = ventas_mes_Estado.reset_index(inplace=False)

  #realizo grafico
  fig.add_trace(go.Scatter(
    x=ventas_mes_Estado['Estado'],
    y=ventas_mes_Estado['Cantidad Pedida'],
    mode='lines',
    name=mes,

    line=dict(shape='spline', smoothing=1.1, width=2)))

  # Configuro los ejes y los títulos
fig.update_layout(
    title=" Ventas por Estado Vs Meses",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Unidades vendidas", side="left"),

)

fig.show()

In [ ]:
fig= go.Figure()#creo un objeto figura
ventas_meses = pd.DataFrame()#creo un dataframe vacio
#realizo una iteracion en la cual agrupo la cantidad pedida de acuerdo a su ciudad para cada mes
for mes in nombres_meses:
  dataframe_mes = dataframes[mes]
  ventas_mes = dataframe_mes.groupby('Ciudad')['Cantidad Pedida'].sum()
  ventas_mes = ventas_mes.reset_index(inplace=False)
  #genero un nuevo dataframe en la cual acumulo la cantidad pedida en los distintos meses
  ventas_meses['Ciudad']=ventas_mes['Ciudad']
  ventas_meses[mes] = ventas_mes['Cantidad Pedida']
#al finalizar la iteracion realizo la suma de todos los meses para tener el total anual en la cantidad pedida por año
ventas_meses['total']=(ventas_meses['Enero']+ventas_meses['Febrero']+ventas_meses['Marzo']+ventas_meses['Abril']+ventas_meses['Mayo']+ventas_meses['Junio']+ventas_meses['Julio']+ventas_meses['Agosto']+ventas_meses['Septiembre']+ventas_meses['Octubre']+ventas_meses['Diciembre']+ventas_meses['Noviembre'])
#realizo el grafico de barras
fig.add_trace(go.Bar(
  x=ventas_meses['Ciudad'],
  y=ventas_meses['total'],
  text=ventas_meses['total'],
  marker = dict(color =  ventas_meses['total'], colorscale = 'RdYlGn', colorbar = dict( title = "Cantidad Pedida anual", x = 1.0, thickness = 15) ),))

fig.update_layout(
    title=" Ventas por Ciudad Vs Año",
    xaxis=dict(title="Ciudades"),
    yaxis=dict(title="Unidades vendidas", side="left"),

)
fig.show()

● ¿En qué ciudades se han registrado las mayores ventas?

  rta: se puede observar que las ciudades con mayores ventas fueron San Francisco y Los Angeles. San Francisco supera en mayor medida al resto de las demas ciudades.

● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  rta:hay tres estados que estan igules en comparacion al resto (MA,TX,NY),
  el estado de CA es el que mas ventas tiene con amplia diferencia en los otros estados.

● ¿Existe variación en las ventas por estado a lo largo de los meses?

  rta:se puede observar que en enero las ventas en todas las regiones son bajas, y en diciembre las epocas festivas se observa el pico mas alto de ventas en las todas regiones, Seguido del mes de octubre que esta por debajo de las ventas de Diciembre.

## 4. Análisis del producto más vendido:


In [ ]:
venta_Producto = []

for mes in nombres_meses:
  dataframe_mes = dataframes[mes]
  venta_Producto.append(dataframe_mes.groupby('Producto')['Cantidad Pedida'].sum()) # agrupo cantidades Pedidas por producto

Ventas_por_Producto = pd.DataFrame(venta_Producto, index=nombres_meses)# genera un dataframe con los productos y sus ventas por mes

#constato resultados ejecutar la linea siguiente:
#Ventas_por_Producto

In [ ]:
productos = Ventas_por_Producto.columns
ventas_anuales_productos = [] #lista que guarda la suma de las ventas anules de cada poducto

for producto in productos:
  ventas_anuales_productos.append(Ventas_por_Producto[producto].sum())

# grafico Ventas Anuales por Producto

fig = go.Figure()
fig.add_trace(go.Bar(
    x = productos,
    y = ventas_anuales_productos,
    marker = dict(color =  ventas_anuales_productos, colorscale = 'RdYlGn', colorbar = dict( title = "Ventas", x = 1.0, thickness = 5) ),
    text =  ventas_anuales_productos,
    textposition = 'auto',
))

# configuracion de los ejes y titulos

fig.update_layout(
    title =" Ventas anuales por Productos",
    xaxis = dict( title = "Productos" ),
    yaxis = dict( title = "Ventas Totales"),
)


fig.show()

In [ ]:
ventas=[] # serie que guarda las ventas mensuales para cada Producto

fig = go.Figure()

for producto in Ventas_por_Producto.columns:
  ventas=(Ventas_por_Producto[producto])


  fig.add_trace(go.Scatter(
        x=nombres_meses,
        y=ventas,
        mode='lines',
        name=f'{producto} ',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Ventas por mes de los diferentes productos",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Ventas", side="left"),
    legend=dict(title="Productos",y=0.3,yanchor='bottom'),
)

fig.show()

● ¿Cuál es el producto más vendido en general y en cada mes?
Rta:De la grafica "Ventas anuales por producto", se puede apreciar que el producto mas vendido con 31017 unidades, es la Bateria triple A (pack x 4). Este producto lidera la cantidad de ventas a lo largo del año, como se puede apreciar en la grafica de "Ventas por mes de los diferentes productos".

● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?
Rta: Las ventas del pack por 4 de las baterias triple A, se puede deber a diversas razones. Una de las principales es el uso en multiples dipositivos de usos cotidiano.Ya que, la industria de estos dispositivos, cada vez mas utilizan este tipo de baterias por su tamaño, eficiencia energetica e impacto ambiental.

## 5. Tendencia ventas:




¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
en los diferentes meses?

rta: Como se puede ver en el primer grafico no se puede llegar a notar una tendencia ya que a lo largo de los dias de los diferentes meses no se observa algun caso en particular que sea notablemente superior o inferior las ventas a lo largo de los dias.
Ahora se analizara el grafico de mapa de calor el cual agrupando los dias en bloques de 5 dias se podra notar unas tendencias mas notables. En los meses de Enero, Febrero, Marzo y Octubre se puede llegar a observar que las ventas se mantienen muy estables en la cantidad media, entre el maximo y el minimo, durante los dias del mes.
Otro patron observable se puede apreciar en los meses de Junio, Julio, Agosto, Septiembre
donde en casi todos los dias se observa q las ventas se concentraron mayormente en las cantidades maximas para sus respectivos meses. Esto puede ser influenciado por algunas fechas patrias las cuales son: 14 jun. Día de la bandera, 4 jul. Día de la Independencia,11 sept	Fiesta para el 11 de septiembre, 17 sept. Día de la Constitución
Estas son las tendencias mas significativas de patrones de ventas. en los otros meses la lectura es confusa ya que no muestra una tendencia sino que las ventas estan distribuidas entre el minimo y el maximo de cada mes.


● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

rta: se Observa que los dias de lunes a martes hay tenmdencia creciente o igual en 8 meses, consecutivo tambien se mantiene esta obsevarcion en 8 diferentes meses para el miercoles. El jueves se observa que las ventas se mantienen o disminuyen. el viernes tiende a disminuir levemente las ventas.
Los sabados como se puede observar en el grafico hay 6 meses donde las ventas aumentan, llegando a un Domingo con menos ventas en 6 meses diferentes.

● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

rta: los fines de semana durante los meses de Enero, Febrero, Julio, Abril, Mayo, Octubre se mantiene la tendencia de que las ventas disminuyeron respecto a los dias laborales. los meses con mas ventas respecto a los otros dias son Septiembre, junio, Marzo en los cuales se nota una diferencia significativa.

In [ ]:
import plotly.graph_objects as go
figura=go.Figure()
nsimbol=0
for mes in nombres_meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)
    df_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum()#acumulo las ventas respecto a los dias
    df_dia = df_dia.reset_index(inplace=False)
    figura.add_trace(go.Scatter(
        x = df_dia['Dia'] ,
        y = df_dia['Cantidad Pedida'],
        name=mes,
        marker_symbol=str(nsimbol),
        line=dict(width=0.5),
        mode='lines+markers'))
    nsimbol+=1
figura.update_layout(
    title=" Ventas por Dia Vs Cantidad Pedida",
    xaxis=dict(title="Dias"),
    yaxis=dict(title="Cantidad Pedida", side="left"))
figura.show()

In [ ]:
import plotly.graph_objects as go
#importo libreria para hacer subplots
from plotly.subplots import make_subplots
#creo un subplot con 3 filas y 4 columnas
figura = make_subplots(3,4)
nrow=1#variable para filas
ncol=1#variable para columna

for mes in nombres_meses:#recorro el df por los meses
    if ncol > 4:#si la fila ya tiene las 4 columnas
        nrow=nrow+1#entonces cambia de fila
        ncol=1#y resetea la columna
    else:#sino pasa al codigo
        pass
    dataframe_mes = dataframes[mes]#analizo cada mes por separado
    dataframe_mes = extraer_datos(dataframe_mes)#extraigo con la funcion antes definida los dias respectivos
    df_dia = dataframe_mes.groupby('Dia')['Cantidad Pedida'].sum()#acumulo las ventas respecto a los dias
    df_dia = df_dia.reset_index(inplace=False)#reseteo el indice del nuevo dataframe agrupado por dia
    #realizo el grafico de mapa de calor
    figura.add_trace(go.Histogram2d(histfunc='sum',
        name=mes,
        y=df_dia['Dia'],
        x=df_dia['Cantidad Pedida'],
        coloraxis='coloraxis',texttemplate= "%{z}"),row=nrow,col=ncol)
    figura.update_yaxes(dict(title=f"Dias en {mes} "),row=nrow,col=ncol)
    figura.update_xaxes(dict(title="Cantidad Pedida"))
    #actualizo el grafico para que me tome cada 5 dias el agrupamiento
    figura.update_traces(ybins_end=31,ybins_size=5,xbins_size=25)
    #variable para pasar a la columna siguiente
    ncol=ncol+1

figura.update_layout(title_text="Heatmap Cantidad Pedida Subplots",height=800, width=1400)
figura.show()

### Analisis dias en la semana

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
nsimbol=0
figura=go.Figure()
df_mes_dia=pd.DataFrame()
for mes in nombres_meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)
    df_dia_sem = dataframe_mes.groupby('Nombre Dia')['Cantidad Pedida'].sum()#acumulo las ventas respecto a los dias
    df_dia_sem = df_dia_sem.reset_index(inplace=False).sort_values('Nombre Dia', ascending=False)
    df_dia_sem=df_dia_sem.reindex(index=[1,5,6,4,0,2,3])
    figura.add_trace(go.Scatter(name=mes,
                                y=df_dia_sem['Nombre Dia'],
                                x=df_dia_sem['Cantidad Pedida'],
                                marker_symbol=str(nsimbol),
                                line=dict(width=0.8)
                                ))
    nsimbol+=1
figura.update_layout(
    title=" Ventas por Dia de semana Vs Cantidad Pedida",
    xaxis=dict(title="Cantidad Pedida"),
    yaxis=dict(title="Dias", side="left"))
figura.show()

## 6.. Impacto de eventos especiales en las ventas

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

meses_festivos=[ 'Enero','Febrero','Mayo','Junio','Julio','Septiembre','Octubre','Noviembre','Diciembre']


data = {'Fecha': ['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-26', '2019-01-27', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-02', '2019-02-03', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-09', '2019-02-10', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-23', '2019-02-24', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-04', '2019-05-05', '2019-05-06', '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10', '2019-05-11', '2019-05-12', '2019-05-13', '2019-05-14', '2019-05-15', '2019-05-16', '2019-05-17', '2019-05-18', '2019-05-19', '2019-05-20', '2019-05-21', '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-25', '2019-05-26', '2019-05-27', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31', '2019-06-01', '2019-06-02', '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-15', '2019-06-16', '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-22', '2019-06-23', '2019-06-24', '2019-06-25', '2019-06-26', '2019-06-27', '2019-06-28', '2019-06-29', '2019-06-30', '2019-07-01', '2019-07-02', '2019-07-03', '2019-07-04', '2019-07-05', '2019-07-06', '2019-07-07', '2019-07-08', '2019-07-09', '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-13', '2019-07-14', '2019-07-15', '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19', '2019-07-20', '2019-07-21', '2019-07-22', '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-27', '2019-07-28', '2019-07-29', '2019-07-30', '2019-07-31', '2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08', '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16', '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20', '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28', '2019-09-29', '2019-09-30', '2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04', '2019-10-05', '2019-10-06', '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-12', '2019-10-13', '2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-19', '2019-10-20', '2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25', '2019-10-26', '2019-10-27', '2019-10-28', '2019-10-29', '2019-10-30', '2019-10-31', '2019-11-01', '2019-11-02', '2019-11-03', '2019-11-04', '2019-11-05', '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-09', '2019-11-10', '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15', '2019-11-16', '2019-11-17', '2019-11-18', '2019-11-19', '2019-11-20', '2019-11-21', '2019-11-22', '2019-11-23', '2019-11-24', '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-28', '2019-11-29', '2019-11-30', '2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-07', '2019-12-08', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-14', '2019-12-15', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-21', '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
        'Cantidad Pedida': [343, 368, 330, 330, 355, 319, 343, 318, 357, 368, 363, 340, 368, 324, 358, 349, 327, 355, 364, 385, 346, 359, 325, 391, 338, 423, 343, 368, 358, 339, 308, 455, 499, 447, 452, 432, 452, 474, 489, 494, 506, 478, 498, 543, 466, 447, 493, 510, 508, 447, 500, 455, 507, 449, 531, 467, 485, 509, 450, 541, 578, 550, 595, 607, 567, 636, 606, 641, 603, 558, 634, 606, 675, 568, 615, 635, 542, 637, 644, 588, 652, 590, 586, 620, 585, 569, 632, 618, 625, 535, 500, 519, 485, 523, 530, 542, 467, 501, 489, 507, 510, 528, 511, 536, 537, 529, 501, 497, 538, 528, 529, 446, 455, 489, 500, 531, 449, 510, 548, 499, 521, 519, 519, 506, 516, 519, 511, 538, 541, 530, 486, 511, 523, 495, 573, 531, 448, 554, 501, 536, 507, 546, 478, 483, 536, 557, 497, 532, 489, 508, 540, 425, 452, 433, 444, 406, 433, 415, 425, 442, 443, 456, 450, 427, 452, 462, 419, 454, 440, 468, 434, 437, 411, 440, 433, 403, 422, 472, 419, 457, 423, 717, 767, 703, 722, 766, 716, 751, 721, 680, 756, 682, 709, 729, 695, 725, 724, 728, 738, 770, 738, 710, 770, 759, 734, 742, 808, 738, 745, 685, 693, 760, 680, 671, 604, 700, 643, 644, 699, 710, 667, 636, 622, 652, 684, 642, 600, 647, 671, 673, 688, 628, 718, 662, 623, 700, 650, 671, 688, 594, 645, 662, 843, 914, 832, 879, 905, 910, 843, 870, 856, 897, 894, 919, 957, 924, 915, 914, 1044, 870, 942, 912, 875, 874, 988, 908, 902, 925, 869, 931, 954, 928, 886]}


df = pd.DataFrame(data)


# Agregar columnas para los días festivos nacionales de Estados Unidos

dias_festivos = ['2019-01-01', '2019-02-18', '2019-05-27', '2019-06-19','2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11', '2019-12-25']
df['EsFestivo'] = df['Fecha'].isin(dias_festivos)

# Crea tres figuras separadas
fig1 = go.Figure()
fig2 = go.Figure()
fig3 = go.Figure()

# Divide el DataFrame en tres partes

df1 = df[0:90]
df2 = df[90:181]
df3 = df[181:273]


# Crea la gráfica Scatter para cada figura
def create_scatter(fig, df, title):
    fig.add_trace(go.Scatter(x=df['Fecha'], y=df['Cantidad Pedida'], mode='markers',
                             marker=dict(size=df['Cantidad Pedida'] / 40, opacity=0.7),
                             text=df['Fecha'], name='Cant. Pedida'))
    festivos = df[df['EsFestivo']]
    fig.add_trace(go.Scatter(x=festivos['Fecha'], y=festivos['Cantidad Pedida'], mode='markers',
                             marker=dict(size=festivos['Cantidad Pedida'] / 40, color='red'),
                             text=festivos['Fecha'], name='Cant. Pedida Días Festivos'))
    fig.update_layout(title=title)

create_scatter(fig1, df1, 'Cantidad Pedida Enere/Febrero/Mayo')
create_scatter(fig2, df2, 'Cantidad Pedida Junio/Julio/Septiembre')
create_scatter(fig3, df3, 'Cantidad Pedida Octubre/Noviembre/Diciembre')

# Mostrar las tres figuras
fig1.show()
fig2.show()
fig3.show()




In [ ]:
# Calculo de la medida de tendencia central -Mediana- y la medida de dispercion -IQR- de la cantidad de ventas de cada mes

# Agregar una columna 'Mes' para agrupar los datos por mes
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Mes'] = df['Fecha'].dt.month

# resultados compila las medianas y el IQR para cada mes
resultados = []

for mes in range(1, 13):  # Suponiendo que tienes datos para cada mes del año saco la mediana y quantiles
    df_mes = df[df['Mes'] == mes]
    mediana = df_mes['Cantidad Pedida'].median()
    Q1 = df_mes['Cantidad Pedida'].quantile(0.25)
    Q3 = df_mes['Cantidad Pedida'].quantile(0.75)
    IQR = Q3 - Q1
    resultados.append({
        'Mes': mes,
        'Mediana': mediana,
        'Rango Intercuartil (IQR)': IQR
    })

# Creo un nuevo DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)
# Elimino valores no deseados
df_resultados = df_resultados.dropna()
# Mostrar los resultados
print("Medidas de tendencia central y dispersion de las ventas en meses con dias festivos")
df_resultados
df_resultados.to_excel("resultado.xlsx", index=False) #eso me lo convierte a Excel

Medidas de tendencia central y dispersion de las ventas en meses con dias festivos


,Mes,Mediana,Rango Intercuartil (IQR)
0,1,349.0,29.50
1,2,481.5,49.50
4,5,606.0,59.50
5,6,510.0,31.50
6,7,519.0,32.50
8,9,435.5,28.00
9,10,729.0,40.50
10,11,662.0,40.75
11,12,908.0,50.00


● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
cercanos a eventos especiales, como días festivos?

Rta:

Analisis mes a mes:

Enero: Vemos que las ventas en el dia festivo no tuvieron un gran impacto. Se registraron 343 unidades. Mientras que la medida de tendencia central fue 349 unidades. Si se puede apreciar que al dia siguiente las ventas tuvieron un movimiento alsista que duro 24hs.

Febrero: Vemos que las ventas en el dia festivo sostuvieron la tendencia a la baja, que venia del dia anterior y continua al dia siguiente. Pero las cantiades vendidas -508 unidades- en este dia se ubico por encima de la tendencia central de mes -481,5 unidades-.

Mayo: Vemos que, en este mes las ventas en el dia festivo se ubicaron en el grupo de menores unidades vendidas del mes con 569 unidades. Despues de este dia, las ventas que venian en baja, tuvieron un gran crecimiento.


Junio: En este mes se puede apreciar que las unidades vendidas en el dia festivo se ubico como la segunda cantidad mas vendida del mes - con 538 unidades-. Teniendo un gran crecimiento - 538 unidades - en comparacion al dia anterior que fue de solo 497 unidades.


Julio: El 4 de Julio notamos que se produjo una baja abrupta de ventas. Vemos que en los dias cercanos al mismo, las ventas venian con un promedio de 519 unidades por dia, cayendo a 506 unidades el 4 de julio. Igualmente no tuvo gran desviacion con respecto a la tendencia central, que se ubico en 519 unidades.

Septiembre: Del grafico se observa, que en este dia. Hubo un incremento en las ventas respecto a los dias adyacentes. Ubicando este dia por encima de la medida de tendencia central, con 452 unidades respeto a las 435 de la medida de tendencia central.

Octubre: El 14 de Octubre, se puede apreciar que hubo una caida en las ventas que venia de 729 unidades, cayendo a 695 en el dia festivo. Las cuales se repucieron en los dias siguientes. Esta caida en las ventas se ubico en unas de las mas bajas del mes.

Noviembre: Del grafico se puede observar que el dia 11 de Noviembre se ubico dentro de los cuatro dias con cantidad de ventas mas bajas del mes, con 622 unidades. Sin embargo sirvio como punto de inflexion para impulsar una tendencia alsista en los siguientes dias.

Diciembre: El 25 de diciembre se observa una leve disminucion en las ventas con respecto al dia anterior.Luego del 25 de dicho mes, toma una tendencia alsista hasta el 29 de Diciembre. Salvando el 27 que se produce un desplome de ventas llegando a 869 unidades, mucho menos que las 902 unidades del dia 25.Por otra parte, las ventas producidas el dia 25 de Diciembre, no tuvieron gran impacto en comparacion con las demas ventas del mes. Se ubicaron muy cerca a la medida de tendencia central que fue de 908 unidades.

En conclucion, las cantidades de ventas en los dias festivos no tuvieron grandes incidencias sobre las ventas del mes. Salvo los meses de Febrero, Junio y Septiembre. En los cuales se ubicaron dentro de los dias con mayor cantidad de ventas.
En general, se mantuvieron dentro del rango de dispercion calculado con respecto a la medida de tendencia central de cada mes.
Lo que si, se puede apreciar en los graficos la tendencia alsistas de las ventas en el correr del año.



In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


data = {'Fecha': ['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-26', '2019-01-27', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-02', '2019-02-03', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-09', '2019-02-10', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-23', '2019-02-24', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-04', '2019-05-05', '2019-05-06', '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10', '2019-05-11', '2019-05-12', '2019-05-13', '2019-05-14', '2019-05-15', '2019-05-16', '2019-05-17', '2019-05-18', '2019-05-19', '2019-05-20', '2019-05-21', '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-25', '2019-05-26', '2019-05-27', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31', '2019-06-01', '2019-06-02', '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-15', '2019-06-16', '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-22', '2019-06-23', '2019-06-24', '2019-06-25', '2019-06-26', '2019-06-27', '2019-06-28', '2019-06-29', '2019-06-30', '2019-07-01', '2019-07-02', '2019-07-03', '2019-07-04', '2019-07-05', '2019-07-06', '2019-07-07', '2019-07-08', '2019-07-09', '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-13', '2019-07-14', '2019-07-15', '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19', '2019-07-20', '2019-07-21', '2019-07-22', '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-27', '2019-07-28', '2019-07-29', '2019-07-30', '2019-07-31', '2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08', '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16', '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20', '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28', '2019-09-29', '2019-09-30', '2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04', '2019-10-05', '2019-10-06', '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-12', '2019-10-13', '2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-19', '2019-10-20', '2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25', '2019-10-26', '2019-10-27', '2019-10-28', '2019-10-29', '2019-10-30', '2019-10-31', '2019-11-01', '2019-11-02', '2019-11-03', '2019-11-04', '2019-11-05', '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-09', '2019-11-10', '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15', '2019-11-16', '2019-11-17', '2019-11-18', '2019-11-19', '2019-11-20', '2019-11-21', '2019-11-22', '2019-11-23', '2019-11-24', '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-28', '2019-11-29', '2019-11-30', '2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-07', '2019-12-08', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-14', '2019-12-15', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-21', '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
        'Cantidad Pedida': [343, 368, 330, 330, 355, 319, 343, 318, 357, 368, 363, 340, 368, 324, 358, 349, 327, 355, 364, 385, 346, 359, 325, 391, 338, 423, 343, 368, 358, 339, 308, 455, 499, 447, 452, 432, 452, 474, 489, 494, 506, 478, 498, 543, 466, 447, 493, 510, 508, 447, 500, 455, 507, 449, 531, 467, 485, 509, 450, 541, 578, 550, 595, 607, 567, 636, 606, 641, 603, 558, 634, 606, 675, 568, 615, 635, 542, 637, 644, 588, 652, 590, 586, 620, 585, 569, 632, 618, 625, 535, 500, 519, 485, 523, 530, 542, 467, 501, 489, 507, 510, 528, 511, 536, 537, 529, 501, 497, 538, 528, 529, 446, 455, 489, 500, 531, 449, 510, 548, 499, 521, 519, 519, 506, 516, 519, 511, 538, 541, 530, 486, 511, 523, 495, 573, 531, 448, 554, 501, 536, 507, 546, 478, 483, 536, 557, 497, 532, 489, 508, 540, 425, 452, 433, 444, 406, 433, 415, 425, 442, 443, 456, 450, 427, 452, 462, 419, 454, 440, 468, 434, 437, 411, 440, 433, 403, 422, 472, 419, 457, 423, 717, 767, 703, 722, 766, 716, 751, 721, 680, 756, 682, 709, 729, 695, 725, 724, 728, 738, 770, 738, 710, 770, 759, 734, 742, 808, 738, 745, 685, 693, 760, 680, 671, 604, 700, 643, 644, 699, 710, 667, 636, 622, 652, 684, 642, 600, 647, 671, 673, 688, 628, 718, 662, 623, 700, 650, 671, 688, 594, 645, 662, 843, 914, 832, 879, 905, 910, 843, 870, 856, 897, 894, 919, 957, 924, 915, 914, 1044, 870, 942, 912, 875, 874, 988, 908, 902, 925, 869, 931, 954, 928, 886]}

df = pd.DataFrame(data)


# Agregar columnas para los días festivos nacionales de Estados Unidos
dias_festivos = ['2019-01-01', '2019-02-18', '2019-05-27', '2019-06-19','2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11', '2019-12-25']
df['EsFestivo'] = df['Fecha'].isin(dias_festivos)

# Crea la gráfica Scatter
fig = px.scatter(df, x='Fecha', y='Cantidad Pedida', title='Cantidad Pedida por Fecha General')
fig.update_traces(marker=dict(size=df['Cantidad Pedida']/50, opacity=0.7), selector=dict(mode='markers'))

# Resalta los días festivos
festivos = df[df['EsFestivo']]
fig.add_trace(go.Scatter(x=festivos['Fecha'], y=festivos['Cantidad Pedida'], mode='markers', marker=dict(size=festivos['Cantidad Pedida']/50, color='red'), name='Días Festivos'))


fig.show()



● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
las ventas y cómo se manifestó ese impacto?

Rta:

En el grafico podemos observar la tendencia alcista que tuvieron las ventas en el año 2019. A principios de año, el gobierno federal de Estados Unidos se cerró parcialmente durante un período récord de 35 días debido a disputas sobre el presupuesto y la financiación del muro fronterizo con México. Esto tuvo un impacto en los trabajadores federales y en la economía en general, el cual se puede observar a simple vista en el grafico, ya que enero fue, con diferencia, el mes con menos ventas del año.

El crecimiento anual tambien se debio en gran parte a las decisiones que tuvo la Reserva Federal de EE. UU., que propuso un plan de reduccion de intereses que comenzaron en julio y continuaron en Septiembre y Octubre -como se ve claramente en el grafico-. Ademas, a lo largo de 2019, la tasa de desempleo en Estados Unidos se mantuvo en niveles históricamente bajos, de la mano de una reforma fiscal aprobada un tiempo antes, que hizo que las empresas aumentaron sus inversiones y contrataciones. Esto fue considerado como una señal positiva para la economía y por consecuente tambien en las ventas, ya que indicaba un mercado laboral fuerte y una mayor demanda de trabajadores.

Sin embargo, este crecimiento, se vio alterado por diferentes eventos o conflictos internacionales. Como la Guerra comercial con China que involucró aranceles y sanciones mutuas.
Si bien, este conflicto venia del 2018, escalono significativamente a partir de mayo del 2019.
Sumado a esto, podemos observar el impacto del aumento del conflicto entre EE.UU. e Iran. Que involucro ataques a petroleros en el golfo de Oman. Derivando asi, en una reduccion en la cantidad de venta en los meses de Junio y Septiembre.

